In [ ]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 43.0 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uninstalling spacy-3.4.4:
      Successfully uninstalled spacy-3.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.5.1 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np

import spacy
nlp = spacy.load('en_core_web_sm')

from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import naive_bayes,svm
from sklearn.metrics import accuracy_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/usr/local/lib/python3.9/dist-packages/spacy/util.py:887: UserWarning: [W095] Model 'en_core_web_sm' (3.4.1) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.5.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
pd.set_option('display.max_colwidth',200)

In [ ]:
corpus= pd.read_csv("/content/SMSSpamCollection.csv",delimiter="\t",encoding='utf=8',header=None)

In [ ]:
corpus.head()

,0,1
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


check for data imbalance

In [ ]:
corpus.columns=['target','SMS']

In [ ]:
corpus.head()

,target,SMS
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [ ]:
corpus.target.value_counts(True)*100

ham     86.593683
spam    13.406317
Name: target, dtype: float64

**text preprocessing**
*  tokenization
*  remove stopwords
*   convert tokens to lemmes




TOKENIZATION

In [ ]:
string = 'We have data imbalance.86% of the SMSs are not spam.SO possible metric is recall having said  '


In [ ]:
doc = nlp(string)

In [ ]:
tokens  = [token.text for token in doc]                #upper to lower

In [ ]:
tokens = [token.lower() for token in tokens]

In [ ]:
stop_words = list(STOP_WORDS)                   #remove stopwords

In [ ]:
tokens = [token for token in tokens if token not in stop_words ]

In [ ]:
tokens

['data',
 'imbalance.86',
 '%',
 'smss',
 'spam',
 '.',
 'possible',
 'metric',
 'recall',
 'having',
 'said',
 ' ']

In [ ]:
tokens_string = ' '.join(tokens)     #join as sentence

In [ ]:
doc1  = nlp(tokens_string)

In [ ]:
lemmas = [token.lemma_ for token in doc1 if token.is_alpha == True]    #lemmas

In [ ]:
lemmas


['data', 'smss', 'spam', 'possible', 'metric', 'recall', 'having', 'say']

In [ ]:
def get_lemmas_from_string(string):
  doc = nlp(string)
  tokens = [token.text for token in doc]   #create tokens for string
  tokens = [token.lower() for token in tokens]     #removes upper case
  stop_words  = list(STOP_WORDS)
  tokens = [token for token in tokens if token not in stop_words ]
  tokens_string = ' '.join(tokens)
  doc1 = nlp(tokens_string)
  lemmas = [token.lemma_ for token in doc1 if token.is_alpha == True]
  lemmas_string = ' '.join(lemmas)
  return lemmas_string

In [ ]:
corpus['SMS Lemma'] = corpus['SMS'].apply(get_lemmas_from_string)                           

In [ ]:
corpus.head()

,target,SMS,SMS Lemma
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",jurong point crazy available bugis n great world la e buffet cine get amore wat
1,ham,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,free entry wkly comp win fa cup final tkts text fa receive entry txt apply
3,ham,U dun say so early hor... U c already then say...,u dun early hor u c
4,ham,"Nah I don't think he goes to usf, he lives around here though",nah think go usf live


In [ ]:
text = 'U C iam'

In [ ]:
def remove_single_letter(text):
   test = text.split()                                                     #.split is check  and delte single letter and space
   final = [ word for word in test if len(word)>1]
   final_test = ' '.join(final)
   return final_test
  

In [ ]:
remove_single_letter(text)

'iam'

In [ ]:
corpus['SMS_final'] = corpus['SMS Lemma'].apply(remove_single_letter)

In [ ]:
corpus.head()

,target,SMS,SMS Lemma,SMS_final
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",jurong point crazy available bugis n great world la e buffet cine get amore wat,jurong point crazy available bugis great world la buffet cine get amore wat
1,ham,Ok lar... Joking wif u oni...,ok lar joke wif u oni,ok lar joke wif oni
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,free entry wkly comp win fa cup final tkts text fa receive entry txt apply,free entry wkly comp win fa cup final tkts text fa receive entry txt apply
3,ham,U dun say so early hor... U c already then say...,u dun early hor u c,dun early hor
4,ham,"Nah I don't think he goes to usf, he lives around here though",nah think go usf live,nah think go usf live


In [ ]:
train_data = corpus[['target','SMS_final']]

In [ ]:
train_data.head()

,target,SMS_final
0,ham,jurong point crazy available bugis great world la buffet cine get amore wat
1,ham,ok lar joke wif oni
2,spam,free entry wkly comp win fa cup final tkts text fa receive entry txt apply
3,ham,dun early hor
4,ham,nah think go usf live


In [ ]:
x = train_data['SMS_final']
y  = train_data['target']

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3, random_state= 123)

In [ ]:
stop_words = list(stop_words) + ['aa','aah','da','lar','ok']

In [ ]:
tf_idf = TfidfVectorizer(stop_words = stop_words , max_df = 0.8)           #convert doc into vector tfidfvectorizer

In [ ]:
tf_idf.fit(x_train)

TfidfVectorizer(max_df=0.8,
                stop_words=['never', 'mostly', 'everything', 'some',
                            'beforehand', 'alone', 'only', 'anything', 'five',
                            'although', 'of', 'why', 'over', 'still', 'twelve',
                            'formerly', 'becomes', 'yours', 'while', 'own',
                            'my', 'on', 'us', 'here', 'noone', 'which', 'two',
                            'thence', 'though', 'sixty', ...])

In [ ]:
x_train_df = tf_idf.transform(x_train)
x_test_df = tf_idf.transform(x_test)                                   #sparse matrix token1 = (docfq) --(docno , terfq).......

In [ ]:
x_train_df = x_train_df.todense()
x_test_df = x_test_df.todense()

In [ ]:
x_train_df = pd.DataFrame(x_train_df,columns = tf_idf.get_feature_names_out())
x_test_df = pd.DataFrame(x_test_df,columns = tf_idf.get_feature_names_out())

In [ ]:
y_train = y_train.astype('category')
y_test = y_test.astype('category')

In [ ]:
x_train_df.head()

,aaooooright,aathi,ab,abdoman,abel,aberdeen,abi,ability,abiola,abj,...,yup,zac,zaher,ze,zero,zhong,zindgi,zoe,zogtorius,zyada
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**model** **building**

**NB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()
naive=model.fit(x_train_df,y_train)

In [ ]:
nbtest=naive.predict(x_test_df)
nbtrain=naive.predict(x_train_df)

In [ ]:
from sklearn.metrics import confusion_matrix , classification_report

In [ ]:
print(confusion_matrix(nbtest,y_test))
print(confusion_matrix(nbtrain,y_train))
print(classification_report(nbtest,y_test))
print(classification_report(nbtrain,y_train))

[[1198   36]
 [ 244  194]]
[[3041    0]
 [ 342  517]]
              precision    recall  f1-score   support

         ham       0.83      0.97      0.90      1234
        spam       0.84      0.44      0.58       438

    accuracy                           0.83      1672
   macro avg       0.84      0.71      0.74      1672
weighted avg       0.83      0.83      0.81      1672

              precision    recall  f1-score   support

         ham       0.90      1.00      0.95      3041
        spam       1.00      0.60      0.75       859

    accuracy                           0.91      3900
   macro avg       0.95      0.80      0.85      3900
weighted avg       0.92      0.91      0.90      3900



**ANN**

In [ ]:
y_train_df = [0 if val == 'ham' else 1 for val in y_train]
y_test_df = [0 if val == 'ham' else 1 for val in y_test]

In [ ]:
y_train_array = np.array(y_train_df)
y_test_array = np.array(y_test_df)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(5024,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train_df, y_train_array, epochs=10, batch_size=64,validation_data=(x_test_df, y_test_array))

Epoch 1/10
61/61 [==============================] - 3s 22ms/step - loss: 0.4918 - accuracy: 0.8654 - val_loss: 0.2740 - val_accuracy: 0.8624
Epoch 2/10
61/61 [==============================] - 1s 13ms/step - loss: 0.1756 - accuracy: 0.9221 - val_loss: 0.1269 - val_accuracy: 0.9755
Epoch 3/10
61/61 [==============================] - 1s 13ms/step - loss: 0.0672 - accuracy: 0.9856 - val_loss: 0.0799 - val_accuracy: 0.9797
Epoch 4/10
61/61 [==============================] - 1s 13ms/step - loss: 0.0289 - accuracy: 0.9933 - val_loss: 0.0762 - val_accuracy: 0.9803
Epoch 5/10
61/61 [==============================] - 1s 13ms/step - loss: 0.0166 - accuracy: 0.9959 - val_loss: 0.0768 - val_accuracy: 0.9797
Epoch 6/10
61/61 [==============================] - 1s 14ms/step - loss: 0.0103 - accuracy: 0.9974 - val_loss: 0.0796 - val_accuracy: 0.9809
Epoch 7/10
61/61 [==============================] - 1s 13ms/step - loss: 0.0072 - accuracy: 0.9990 - val_loss: 0.0821 - val_accuracy: 0.9809
Epoch 8/10
61

In [ ]:
test_loss, test_acc = model.evaluate(x_test_df, y_test_array)
print('Test accuracy:', test_acc)

53/53 [==============================] - 0s 4ms/step - loss: 0.0903 - accuracy: 0.9803
Test accuracy: 0.9802631735801697
